In [20]:
from gurobipy import *
import pandas as pd
import numpy as np

## Parameters

In [21]:
df_county = pd.read_csv('data/county_code.csv')
df_market = pd.read_csv('data/market_code.csv')
I = range(df_county.shape[0])
J = range(df_market.shape[0])

In [43]:
df_demand = pd.read_excel("data/demand_function.xlsx")
A = df_demand["slope"]
B = df_demand["intercept"]
# df_demand

In [65]:
# 旺季產量
df_production = pd.read_csv("data/county_production.csv")
# 旺季一個月
df_production1 = df_production.copy()
df_production1['quantity'] = df_production1['quantity'] * 0.6 / 4 
# 每天
df_production1['quantity'] /= 30
Q_s = df_production1['quantity']
df_production1

,Unnamed: 0,name,quantity,code
0,0,新北市,15366.243889,0
1,1,台北市,2748.499444,1
2,2,桃園市,16033.365000,2
3,3,台中市,176370.740556,3
4,4,台南市,95918.435556,4
5,5,高雄市,38387.124444,5
6,6,宜蘭縣,431689.425000,6
7,7,新竹縣,37136.298333,7
8,8,苗栗縣,23024.741667,8
9,9,彰化縣,226593.761111,9


In [66]:
L = 2500
C = 1.53
M = 6800

In [67]:
Q_s

0      15366.243889
1       2748.499444
2      16033.365000
3     176370.740556
4      95918.435556
5      38387.124444
6     431689.425000
7      37136.298333
8      23024.741667
9     226593.761111
10    233545.821111
11    458767.212222
12    117518.770556
13     74353.701667
14     31505.105000
15     19572.552778
16        23.720000
17       603.262222
18      1003.147222
19     12541.462500
Name: quantity, dtype: float64

In [68]:
Q_s.sum()

2012703.3902777776

- Formulation1
$$
\begin{split}
 \mbox{max} \quad & \sum_{j \in J}p_j\sum_{i \in I}x_{ij} - C\sum_{i \in I}\sum_{j \in J}(My_{ij} + x_{ij}) \quad \mbox{(revenue)} \\
 \mbox{s.t.} \quad & \sum_j x_{ij} \leq Q^s_i \quad \forall i \in I \quad \mbox{(upper limit of production quantity)}\\
 & p_j = A_j \sum_{i \in I} x_{ij} + B_j \quad \forall j \in J \\
 & x_{ij} \leq y_{ij}L \quad \forall i \in I, j \in J \quad \mbox{(compute number of truck)}\\
 & x_{ij} \geq 0 \quad \forall i \in I, j \in J \quad \mbox{(Sign Constraint)} \\
 & y_{ij} \in \mathbb{Z}^+ \quad \forall i \in I, j \in J \quad \mbox{(Sign Constraint)} \\
\end{split}
$$

In [95]:
model = Model('solution1')
model.setParam('NonConvex', 2)

# decision variables
x = model.addVars(I, J, lb = 0, vtype = GRB.CONTINUOUS, name = 'x')
y = model.addVars(I, J, lb = 0, vtype = GRB.INTEGER, name = 'y')
p = model.addVars(J, lb = 0, vtype = GRB.CONTINUOUS, name = 'p')

# objective function
model.setObjective(quicksum(p[j] * quicksum(x[i, j] for i in I) for j in J) 
                            - C * quicksum(M * y[i, j] + x[i, j] for i in I for j in J), GRB.MAXIMIZE)

# constraints (for both single and multiple sourcing)
constr = model.addConstrs(quicksum(x[i, j] for j in J) <= Q_s[i] for i in I)
constr = model.addConstrs(x[i, j] <= y[i, j] * L for i in I for j in J)
constr = model.addConstrs(p[j] == A[j] * quicksum(x[i, j] for i in I) + B[j] for j in J)

model.optimize()

Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 377 rows, 697 columns and 1377 nonzeros
Model fingerprint: 0xd7ba32c7
Model has 340 quadratic objective terms
Variable types: 357 continuous, 340 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [2e+00, 1e+04]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 5e+05]
Found heuristic solution: objective -7.07472e+15
Presolve removed 17 rows and 0 columns
Presolve time: 0.01s
Presolved: 1041 rows, 1038 columns, 2789 nonzeros
Presolved model has 340 bilinear constraint(s)
Found heuristic solution: objective -0.0000000
Variable types: 698 continuous, 340 integer (340 binary)

Root relaxation: objective 1.223635e+08, 1070 iterations, 0.01 seconds (0.01 work units)

Interrupt request 

In [96]:
for var in model.getVars():
    if var.varName.startswith('x'):
        print(var.varName, '=', var.x)
    elif var.varName.startswith('y'):
        print(var.varName, '=', var.x)
    elif var.varName.startswith('p'):
        print(var.varName, '=', var.x)

x[0,0] = 0.0
x[0,1] = 0.0
x[0,2] = 0.0
x[0,3] = 0.0
x[0,4] = 0.0
x[0,5] = 0.0
x[0,6] = 0.0
x[0,7] = 0.0
x[0,8] = 0.0
x[0,9] = 0.0
x[0,10] = 0.0
x[0,11] = 0.0
x[0,12] = 0.0
x[0,13] = 0.0
x[0,14] = 80.31656565407731
x[0,15] = 0.0
x[0,16] = 30.23806751539769
x[1,0] = 0.0
x[1,1] = 0.0
x[1,2] = 0.0
x[1,3] = 0.0
x[1,4] = 0.0
x[1,5] = 0.0
x[1,6] = 0.0
x[1,7] = 0.0
x[1,8] = 0.0
x[1,9] = 0.0
x[1,10] = 0.0
x[1,11] = 0.0
x[1,12] = 0.0
x[1,13] = 0.0
x[1,14] = 0.0
x[1,15] = 0.0
x[1,16] = 0.0
x[2,0] = 0.0
x[2,1] = 0.0
x[2,2] = 0.0
x[2,3] = 0.0
x[2,4] = 0.0
x[2,5] = 0.0
x[2,6] = 0.0
x[2,7] = 0.0
x[2,8] = 0.0
x[2,9] = 0.0
x[2,10] = 0.0
x[2,11] = 0.0
x[2,12] = 0.0
x[2,13] = 0.0
x[2,14] = 0.0
x[2,15] = 0.0
x[2,16] = 0.0
x[3,0] = 0.0
x[3,1] = 0.0
x[3,2] = 0.0
x[3,3] = 0.0
x[3,4] = 0.0
x[3,5] = 0.0
x[3,6] = 0.0
x[3,7] = 0.0
x[3,8] = 0.0
x[3,9] = 0.0
x[3,10] = 0.0
x[3,11] = 0.0
x[3,12] = 0.0
x[3,13] = 0.0
x[3,14] = 0.0
x[3,15] = 0.0
x[3,16] = 0.0
x[4,0] = 0.0
x[4,1] = 0.0
x[4,2] = 0.0
x[4,3] = 0.0
x[4,4] =

In [97]:
df_result_production = df_county.copy()
p = []
for i in I:
    total = 0
    for j in J:
        total += x[i, j].x
    p.append(total)

In [98]:
p

[110.554633169475,
 0.0,
 0.0,
 0.0,
 59.7697726954445,
 35.50096311271497,
 0.0,
 0.0,
 22.192524928118548,
 26.412357326682272,
 89.51374156732186,
 0.0,
 0.0,
 0.0,
 0.0,
 67.00780870043248,
 21.763593633910105,
 12.3938606171853,
 36.377329802690966,
 105.02749614774173]

In [30]:
df_demand

,Unnamed: 0,ID,slope,intercept
0,台北二,104,-2827.400,200752.0
1,台北一,109,-6172.900,449106.0
2,三重區,241,-1980.800,85186.0
3,桃農,338,-2504.800,109048.0
4,台中市,400,-1548.200,68717.0
5,永靖鄉,512,-510.470,31958.0
6,溪湖鎮,514,-7079.600,175488.0
7,南投市場,540,-48.848,2784.9
8,西螺鎮,648,-6081.000,460301.0
9,高雄市,800,-2367.000,125038.0


In [51]:
# consumption
df_consumption = pd.read_csv("data/consumption.csv")
df_consumption = df_consumption.replace("南投市", "南投市場")
df_consumption = df_consumption[df_consumption["market_name"].isin(list(df_demand["Unnamed: 0"]))]
df_consumption["month"] = df_consumption["date"].str.split('.').str[1]
consumptions = []
month = df_consumption["month"].unique()
for i in month:
    df = df_consumption[df_consumption["month"] == i][["market_name", 
                        "quantity", "mean_price"]].groupby(["market_name"]).mean()
    df = df_demand[["Unnamed: 0"]].merge(right = df, right_on = "market_name", left_on = "Unnamed: 0")
    consumptions.append(df)

print(consumptions[0])

   Unnamed: 0       quantity  mean_price
0         台北二  157183.655977   13.280257
1         台北一  378746.411079   13.322492
2         三重區   50967.553936   12.080210
3          桃農   32963.338192   12.052526
4         台中市   42682.312865   11.989855
5         永靖鄉   56868.415441    9.967664
6         溪湖鎮   84505.971098   12.446673
7        南投市場    2919.810496   14.902399
8         西螺鎮  379836.636637   11.044560
9         高雄市   79209.460641   15.881483
10        鳳山區   24236.137026   11.331352
11        屏東市   33785.737609   12.140778
12        花蓮市   25497.405917   15.766917
13        台東市   14901.787582   15.333606
14        宜蘭市   20436.245353    9.801510
15        豐原區   40815.417040   10.716375
16        板橋區   88827.083436   12.826775


In [73]:
P = consumptions[0].mean_price
Q_d = consumptions[0].quantity

In [74]:
Q_d.sum()

1514383.380325805

- Formulation1
$$
\begin{split}
 \mbox{min} \quad & C\sum_{i \in I}\sum_{j \in J}(My_{ij} + x_{ij}) \quad \mbox{(revenue)} \\
 \mbox{s.t.} \quad & \sum_j x_{ij} \leq Q^s_i \quad \forall i \in I \quad \mbox{(upper limit of production quantity)}\\
 & \sum_i x_{ij} \leq Q^d_j \quad \forall j \in J \quad \mbox{(lower limit of demand quantity)}\\
 & x_{ij} \leq y_{ij}L \quad \forall i \in I, j \in J \quad \mbox{(compute number of truck)}\\
 & x_{ij} \geq 0 \quad \forall i \in I, j \in J \quad \mbox{(Sign Constraint)} \\
 & y_{ij} \in \mathbb{Z}^+ \quad \forall i \in I, j \in J \quad \mbox{(Sign Constraint)} \\
\end{split}
$$

In [ ]:
# Calcalate directly from real price
model = Model('solution1-1')

# decision variables
x = model.addVars(I, J, lb = 0, vtype = GRB.CONTINUOUS, name = 'x')
y = model.addVars(I, J, lb = 0, vtype = GRB.INTEGER, name = 'y')

# objective function
model.setObjective(C * quicksum(M * y[i, j] + x[i, j] for i in I for j in J), GRB.MINIMIZE)

# constraints (for both single and multiple sourcing)
constr = model.addConstrs(quicksum(x[i, j] for j in J) <= Q_s[i] for i in I)
constr = model.addConstrs(quicksum(x[i, j] for i in I) >= Q_d[j] for j in J)
constr = model.addConstrs(x[i, j] <= y[i, j] * L for i in I for j in J)
# constr = model.addConstrs(p[j] == A[j] * quicksum(x[i, j] for i in I) + B[j] for j in J)

model.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 377 rows, 680 columns and 1360 nonzeros
Model fingerprint: 0x045dc9d8
Variable types: 340 continuous, 340 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [2e+00, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 5e+05]
Found heuristic solution: objective 8819506.5719
Presolve time: 0.00s
Presolved: 377 rows, 680 columns, 1360 nonzeros
Variable types: 340 continuous, 340 integer (51 binary)

Root relaxation: objective 8.619264e+06, 567 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 8619264.45    0   32 8819506.57 8619264.45  2.27%     -    0s
H    0     0                    879869

In [100]:
for var in model.getVars():
    if var.varName.startswith('x'):
        print(var.varName, '=', var.x)

x[0,0] = 0.0
x[0,1] = 0.0
x[0,2] = 0.0
x[0,3] = 0.0
x[0,4] = 0.0
x[0,5] = 0.0
x[0,6] = 15366.24388888889
x[0,7] = 0.0
x[0,8] = 0.0
x[0,9] = 0.0
x[0,10] = 0.0
x[0,11] = 0.0
x[0,12] = 0.0
x[0,13] = 0.0
x[0,14] = 0.0
x[0,15] = 0.0
x[0,16] = 0.0
x[1,0] = 0.0
x[1,1] = 0.0
x[1,2] = 0.0
x[1,3] = 0.0
x[1,4] = 0.0
x[1,5] = 0.0
x[1,6] = 2748.499444444444
x[1,7] = 0.0
x[1,8] = 0.0
x[1,9] = 0.0
x[1,10] = 0.0
x[1,11] = 0.0
x[1,12] = 0.0
x[1,13] = 0.0
x[1,14] = 0.0
x[1,15] = 0.0
x[1,16] = 0.0
x[2,0] = 0.0
x[2,1] = 0.0
x[2,2] = 0.0
x[2,3] = 0.0
x[2,4] = 0.0
x[2,5] = 0.0
x[2,6] = 16033.364999999998
x[2,7] = 0.0
x[2,8] = 0.0
x[2,9] = 0.0
x[2,10] = 0.0
x[2,11] = 0.0
x[2,12] = 0.0
x[2,13] = 0.0
x[2,14] = 0.0
x[2,15] = 0.0
x[2,16] = 0.0
x[3,0] = 157183.65597667638
x[3,1] = 19187.08457887915
x[3,2] = 0.0
x[3,3] = 0.0
x[3,4] = 0.0
x[3,5] = 0.0
x[3,6] = 0.0
x[3,7] = 0.0
x[3,8] = 0.0
x[3,9] = 0.0
x[3,10] = 0.0
x[3,11] = 0.0
x[3,12] = 0.0
x[3,13] = 0.0
x[3,14] = 0.0
x[3,15] = 0.0
x[3,16] = 0.0
x[4,0] = 0.0
x[4

In [101]:
Q_s

0      15366.243889
1       2748.499444
2      16033.365000
3     176370.740556
4      95918.435556
5      38387.124444
6     431689.425000
7      37136.298333
8      23024.741667
9     226593.761111
10    233545.821111
11    458767.212222
12    117518.770556
13     74353.701667
14     31505.105000
15     19572.552778
16        23.720000
17       603.262222
18      1003.147222
19     12541.462500
Name: quantity, dtype: float64

In [102]:
df_result_production = df_county.copy()
p = []
for i in I:
    total = 0
    for j in J:
        total += x[i, j].x
    p.append(total)
df_result_production["production"] = p
df_result_production

,Unnamed: 0,code,name,production
0,0,0,新北市,15366.243889
1,1,1,台北市,2748.499444
2,2,2,桃園市,16033.365000
3,3,3,台中市,176370.740556
4,4,4,台南市,95918.435556
5,5,5,高雄市,38387.124444
6,6,6,宜蘭縣,57199.475219
7,7,7,新竹縣,37136.298333
8,8,8,苗栗縣,23024.741667
9,9,9,彰化縣,226593.761111


In [92]:
Q_d

0     157183.655977
1     378746.411079
2      50967.553936
3      32963.338192
4      42682.312865
5      56868.415441
6      84505.971098
7       2919.810496
8     379836.636637
9      79209.460641
10     24236.137026
11     33785.737609
12     25497.405917
13     14901.787582
14     20436.245353
15     40815.417040
16     88827.083436
Name: quantity, dtype: float64

In [103]:
df_result_consumption = df_market.copy()
c = []
for j in J:
    total = 0
    for i in I:
        total += x[i, j].x
    c.append(total)
df_result_consumption["consumption"] = c
df_result_consumption

,Unnamed: 0,code,name,consumption
0,0,0,台北二綜合農產品批發市場,157183.655977
1,1,1,台北一綜合農產品批發市場,378746.411079
2,2,2,三重區綜合農產品批發市場,50967.553936
3,3,3,桃農綜合農產品批發市場,32963.338192
4,4,4,台中市綜合農產品批發市場,42682.312865
5,5,5,永靖鄉綜合農產品批發市場,56868.415441
6,6,6,溪湖鎮綜合農產品批發市場,84505.971098
7,7,7,南投市綜合農產品批發市場,2919.810496
8,8,8,西螺鎮綜合農產品批發市場,379836.636637
9,9,9,高雄市綜合農產品批發市場,79209.460641
